In [1]:
import tushare as ts
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from dateutil.parser import parse
from dateutil.relativedelta import relativedelta
import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf

In [2]:
research_starttime='2010-04-01'
research_endtime='2018-04-01'
format='%Y-%m-%d %H:%M:%S'
data_type=""#"net"
data_mode=""#"fast"
data_mode_global=data_mode


In [6]:
#转换索引
def trans_index(df):
    if(type(df.index[0])==str):
        df.index=pd.to_datetime(df.index)
    return df
        
#get_all_instruments
def all_instruments_cs():
    df_market=None
    if(data_type=="net"):
        df_market = all_instruments(type='CS')
    else:
        df_market=pd.read_csv("all_instruments.csv",index_col=0)
    return df_market

#get_price_cs
def get_price_cs(data_starttime,data_endtime):
    df_price_data=None
    df_market=all_instruments_cs()
    if(data_type=="net"):
        df_price_data = get_price(list(df_market['order_book_id']),start_date=data_starttime, end_date=data_endtime,frequency='1d',fields='close')
    else:
        df_price_data=pd.read_csv("price_data.csv",index_col=0)
        df_price_data=trans_index(df_price_data)
    return df_price_data

#转换Series
def trans_series(df,key=1):
    if(type(df)==pd.DataFrame):
        df=df[key]
    return df

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [21]:
df_book_value=pd.read_csv("./book_value_per_share.csv",index_col=0)
df_market_cap=pd.read_csv("./market_cap.csv",index_col=0)
df_return_on_equity=pd.read_csv("./return_on_equity.csv",index_col=0)
df_total_assets=pd.read_csv("./total_assets.csv",index_col=0)

df_book_value=trans_index(df_book_value)
df_market_cap=trans_index(df_market_cap)
df_return_on_equity=trans_index(df_return_on_equity)    
df_total_assets=trans_index(df_total_assets) 

In [22]:
index_date = df_book_value.index
stocks = df_book_value.columns
data_count=len(index_date)
data_starttime=index_date[0]
data_endtime=index_date[-1]
data_starttime,data_endtime

(Timestamp('2010-04-02 00:00:00'), Timestamp('2018-03-30 00:00:00'))

In [24]:
arr_ydyl=['000022.XSHE', '000042.XSHE', '000065.XSHE', '000088.XSHE', '000090.XSHE', '000151.XSHE', '000157.XSHE', '000159.XSHE', '000400.XSHE', '000425.XSHE', '000498.XSHE', '000507.XSHE', '000528.XSHE', '000564.XSHE', '000582.XSHE', '000610.XSHE', '000617.XSHE', '000672.XSHE', '000680.XSHE', '000703.XSHE', '000777.XSHE', '000797.XSHE', '000852.XSHE', '000862.XSHE', '000877.XSHE', '000905.XSHE', '000928.XSHE', '000939.XSHE', '000978.XSHE', '600026.XSHG', '600028.XSHG', '600031.XSHG', '600068.XSHG', '600089.XSHG', '600105.XSHG', '600118.XSHG', '600125.XSHG', '600150.XSHG', '600170.XSHG', '600176.XSHG', '600179.XSHG', '600190.XSHG', '600202.XSHG', '600256.XSHG', '600279.XSHG', '600312.XSHG', '600317.XSHG', '600320.XSHG', '600339.XSHG', '600350.XSHG', '600368.XSHG', '600406.XSHG', '600425.XSHG', '600428.XSHG', '600449.XSHG', '600477.XSHG', '600487.XSHG', '600495.XSHG', '600509.XSHG', '600522.XSHG', '600528.XSHG', '600540.XSHG', '600580.XSHG', '600581.XSHG', '600583.XSHG', '600706.XSHG', '600717.XSHG', '600720.XSHG', '600778.XSHG', '600798.XSHG', '600801.XSHG', '600820.XSHG', '600888.XSHG', '600984.XSHG', '002040.XSHE', '600970.XSHG', '002047.XSHE', '002051.XSHE', '600017.XSHG', '600018.XSHG', '601872.XSHG', '601008.XSHG', '002135.XSHE', '601919.XSHG', '002146.XSHE', '601808.XSHG', '601857.XSHG', '601390.XSHG', '601866.XSHG', '002205.XSHE', '002207.XSHE', '601186.XSHG', '601668.XSHG', '002266.XSHE', '601766.XSHG', '601727.XSHG', '601989.XSHG', '002302.XSHE', '002307.XSHE', '002309.XSHE', '601618.XSHG', '601117.XSHG', '300011.XSHE', '002323.XSHE', '002353.XSHE', '601179.XSHG', '601106.XSHG', '002459.XSHE', '300090.XSHE', '300103.XSHE', '601018.XSHG', '002475.XSHE', '601028.XSHG', '002554.XSHE', '002738.XSHE', '601880.XSHG', '002524.XSHE', '300183.XSHE', '601789.XSHG', '300208.XSHE', '300262.XSHE', '601669.XSHG', '601800.XSHG', '603333.XSHG', '603308.XSHG', '300351.XSHE', '603111.XSHG', '600023.XSHG', '300523.XSHE', '300374.XSHE', '603969.XSHG', '603569.XSHG', '603338.XSHG', '603018.XSHG', '603169.XSHG', '002828.XSHE', '603036.XSHG', '603298.XSHG', '603966.XSHG', '603218.XSHG', '601228.XSHG', '600326.XSHG', '603588.XSHG', '603619.XSHG', '600219.XSHG', '300065.XSHE', '002742.XSHE', '600415.XSHG', '603871.XSHG', '300732.XSHE', '600338.XSHG', '600010.XSHG', '600039.XSHG', '601899.XSHG', '002800.XSHE', '601500.XSHG', '300589.XSHE']
if(data_type=="net"):
    arr_ydyl=concept('一带一路')


In [14]:
Rm_Rf=pd.read_csv("./Rm_Rf.csv",index_col=0,header=None)
Rm_Rf=trans_series(trans_index(Rm_Rf))
Rm_Rf

SMB=pd.read_csv("./smb.csv",index_col=0,header=None)
SMB=trans_series(trans_index(SMB))
SMB

HML=pd.read_csv("./hml.csv",index_col=0,header=None)
HML=trans_series(trans_index(HML))
HML

CMA=pd.read_csv("./cma.csv",index_col=0,header=None)
CMA=trans_series(trans_index(CMA))
CMA

RMW=pd.read_csv("./rmw.csv",index_col=0,header=None)
RMW=trans_series(trans_index(RMW))
RMW

Exchange=pd.read_csv("./exchange_rate.csv",index_col=0,header=None)
Exchange=trans_series(trans_index(Exchange))
Exchange

y=pd.read_csv("./y.csv",index_col=0,header=None)
y=trans_series(trans_index(y))
y

0
2010-04-02         NaN
2010-05-04   -0.115266
2010-06-04   -0.092520
2010-07-06   -0.067556
2010-08-06    0.129191
2010-09-08    0.027325
2010-10-08    0.019798
2010-11-10    0.147831
2010-12-10   -0.097940
2011-01-11   -0.013594
2011-02-11   -0.003436
2011-03-11    0.038340
2011-04-13    0.036011
2011-05-13   -0.074717
2011-06-15   -0.055114
2011-07-15    0.053361
2011-08-16   -0.076510
2011-09-16   -0.059042
2011-10-18   -0.054441
2011-11-18    0.002928
2011-12-20   -0.090603
2012-01-20    0.050849
2012-02-29    0.049355
2012-03-30   -0.070630
2012-04-11    0.023950
2012-05-11    0.043797
2012-06-13   -0.023718
2012-07-13   -0.052543
2012-08-14   -0.040368
2012-09-14   -0.019763
                ...   
2015-10-30    0.102467
2015-11-30    0.008232
2015-12-31    0.045234
2016-02-01   -0.223365
2016-03-01    0.009302
2016-04-01    0.098446
2016-04-06    0.010143
2016-05-06   -0.039956
2016-06-08    0.009827
2016-07-12    0.033595
2016-08-12    0.005515
2016-09-14   -0.017765
2016-10-1

0
2010-04-02    0.000000
2010-05-04    0.072521
2010-06-04    0.063869
2010-07-06    0.033867
2010-08-06   -0.090692
2010-09-08   -0.017744
2010-10-08   -0.024794
2010-11-10   -0.130030
2010-12-10    0.060862
2011-01-11    0.005458
2011-02-11   -0.007782
2011-03-11   -0.030101
2011-04-13   -0.033197
2011-05-13    0.041192
2011-06-15    0.032963
2011-07-15   -0.051744
2011-08-16    0.049584
2011-09-16    0.036984
2011-10-18    0.024557
2011-11-18   -0.013176
2011-12-20    0.050213
2012-01-20   -0.049039
2012-02-29   -0.039565
2012-03-30    0.047447
2012-04-11   -0.014933
2012-05-11   -0.033474
2012-06-13    0.009621
2012-07-13    0.025162
2012-08-14    0.011435
2012-09-14    0.007234
                ...   
2015-10-30   -0.089938
2015-11-30   -0.025595
2015-12-31   -0.025909
2016-02-01    0.151607
2016-03-01   -0.014541
2016-04-01   -0.075211
2016-04-06   -0.010317
2016-05-06    0.028574
2016-06-08   -0.018613
2016-07-12   -0.044721
2016-08-12   -0.011503
2016-09-14    0.011056
2016-10-1

0
2010-04-02    0.000000
2010-05-04   -0.071589
2010-06-04   -0.049275
2010-07-06   -0.005557
2010-08-06    0.015207
2010-09-08   -0.032602
2010-10-08   -0.031514
2010-11-10    0.018759
2010-12-10   -0.029972
2011-01-11    0.007826
2011-02-11   -0.008489
2011-03-11   -0.006618
2011-04-13    0.020230
2011-05-13   -0.018868
2011-06-15   -0.027779
2011-07-15   -0.008958
2011-08-16   -0.028497
2011-09-16    0.000335
2011-10-18    0.004735
2011-11-18   -0.003222
2011-12-20    0.009006
2012-01-20   -0.007342
2012-02-29   -0.014373
2012-03-30   -0.007823
2012-04-11   -0.011268
2012-05-11   -0.009420
2012-06-13   -0.018366
2012-07-13   -0.024731
2012-08-14    0.009694
2012-09-14   -0.003212
                ...   
2015-10-30    0.006602
2015-11-30   -0.024249
2015-12-31    0.005068
2016-02-01   -0.086866
2016-03-01    0.004257
2016-04-01    0.035953
2016-04-06   -0.001547
2016-05-06   -0.020999
2016-06-08   -0.006409
2016-07-12    0.006119
2016-08-12    0.016789
2016-09-14   -0.020628
2016-10-1

0
2010-04-02    0.000000
2010-05-04    0.051764
2010-06-04    0.045896
2010-07-06    0.022781
2010-08-06   -0.073459
2010-09-08   -0.016918
2010-10-08   -0.031813
2010-11-10   -0.088691
2010-12-10    0.028092
2011-01-11    0.002075
2011-02-11   -0.003402
2011-03-11   -0.017280
2011-04-13   -0.007224
2011-05-13    0.020267
2011-06-15    0.013084
2011-07-15   -0.058743
2011-08-16    0.005121
2011-09-16    0.034821
2011-10-18    0.027782
2011-11-18    0.000357
2011-12-20    0.013863
2012-01-20   -0.013198
2012-02-29   -0.027667
2012-03-30    0.020256
2012-04-11   -0.009407
2012-05-11   -0.024452
2012-06-13   -0.002426
2012-07-13   -0.003377
2012-08-14    0.008065
2012-09-14    0.005348
                ...   
2015-10-30   -0.047610
2015-11-30   -0.030947
2015-12-31   -0.013603
2016-02-01    0.051122
2016-03-01    0.005240
2016-04-01   -0.035842
2016-04-06   -0.007805
2016-05-06    0.008564
2016-06-08   -0.011418
2016-07-12   -0.003757
2016-08-12   -0.002698
2016-09-14   -0.000429
2016-10-1

0
2010-04-02    0.000000
2010-05-04   -0.059653
2010-06-04   -0.055799
2010-07-06   -0.021232
2010-08-06    0.068714
2010-09-08    0.009074
2010-10-08    0.022023
2010-11-10    0.112015
2010-12-10   -0.047111
2011-01-11   -0.003005
2011-02-11    0.001837
2011-03-11    0.015860
2011-04-13    0.023954
2011-05-13   -0.032229
2011-06-15   -0.024353
2011-07-15    0.031788
2011-08-16   -0.022648
2011-09-16   -0.031021
2011-10-18   -0.014331
2011-11-18    0.006063
2011-12-20   -0.030521
2012-01-20    0.046103
2012-02-29    0.021519
2012-03-30   -0.034879
2012-04-11    0.006767
2012-05-11    0.028260
2012-06-13   -0.013471
2012-07-13   -0.011125
2012-08-14   -0.001020
2012-09-14   -0.007077
                ...   
2015-10-30    0.052309
2015-11-30    0.026915
2015-12-31    0.018101
2016-02-01   -0.110696
2016-03-01   -0.001176
2016-04-01    0.060940
2016-04-06    0.005070
2016-05-06   -0.005344
2016-06-08    0.021405
2016-07-12    0.027043
2016-08-12    0.023506
2016-09-14   -0.006605
2016-10-1

0
2010-04-02         NaN
2010-05-04    0.000132
2010-06-04    0.000322
2010-07-06   -0.007058
2010-08-06   -0.001799
2010-09-08    0.003827
2010-10-08   -0.017986
2010-11-10   -0.005680
2010-12-10    0.003196
2011-01-11   -0.005409
2011-02-11   -0.004155
2011-03-11   -0.002624
2011-04-13   -0.006236
2011-05-13   -0.005434
2011-06-15   -0.002493
2011-07-15   -0.003055
2011-08-16   -0.012256
2011-09-16    0.000031
2011-10-18   -0.000329
2011-11-18   -0.003777
2011-12-20   -0.001542
2012-01-20   -0.002048
2012-02-29   -0.006347
2012-03-30    0.000667
2012-04-11    0.001747
2012-05-11    0.000190
2012-06-13    0.009318
2012-07-13    0.001602
2012-08-14   -0.003198
2012-09-14   -0.006904
                ...   
2015-10-30   -0.006135
2015-11-30    0.012678
2015-12-31    0.014910
2016-02-01    0.013059
2016-03-01   -0.003998
2016-04-01   -0.011218
2016-04-06    0.000355
2016-05-06    0.002963
2016-06-08    0.009600
2016-07-12    0.019200
2016-08-12   -0.007954
2016-09-14    0.005742
2016-10-1

0
2010-04-02   -0.001425
2010-05-04   -0.104910
2010-06-04   -0.105708
2010-07-06   -0.040206
2010-08-06    0.086355
2010-09-08    0.012323
2010-10-08    0.019695
2010-11-10    0.149638
2010-12-10   -0.050064
2011-01-11   -0.004565
2011-02-11    0.039500
2011-03-11    0.022863
2011-04-13    0.021456
2011-05-13   -0.079175
2011-06-15   -0.030041
2011-07-15    0.029993
2011-08-16   -0.096175
2011-09-16   -0.043299
2011-10-18   -0.024110
2011-11-18    0.006063
2011-12-20   -0.065865
2012-01-20    0.060819
2012-02-29    0.038671
2012-03-30   -0.076604
2012-04-11    0.014000
2012-05-11    0.015477
2012-06-13   -0.027477
2012-07-13   -0.057522
2012-08-14   -0.002344
2012-09-14   -0.009384
                ...   
2015-10-30    0.097086
2015-11-30   -0.031374
2015-12-31   -0.008286
2016-02-01   -0.224896
2016-03-01    0.018277
2016-04-01    0.102811
2016-04-06    0.011225
2016-05-06   -0.065993
2016-06-08   -0.006972
2016-07-12    0.042188
2016-08-12   -0.000017
2016-09-14    0.005605
2016-10-1

In [18]:
def get_df_xy(index_date,factors):
    df_xy=pd.DataFrame(factors,index=index_date)
    df_xy = df_xy[1:]
    return df_xy

def get_vif(results):
    import statsmodels.stats.outliers_influence as sso
    from statsmodels.compat import lzip
    x_len=results.model.exog.shape[1]
    vif=[]
    for i in range(x_len):
        vif.append(sso.variance_inflation_factor(results.model.exog, i))
#         print(results.model.exog_names[i],sso.variance_inflation_factor(results.model.exog, i))

    ziped=lzip(results.model.exog_names, vif)
    return ziped

In [23]:
df_xy=get_df_xy(index_date,{"y":y,"Rm_Rf":Rm_Rf,'SMB':SMB,'HML':HML,'RMW':RMW,'CMA':CMA,'Exchange':Exchange})
# df_X=get_df_X(index_date,{"Rm_Rf":Rm_Rf,'SMB':smb,'HML':hml})
df_xy.index.name="date"
df_xy.describe()
# df_xy.corr(method='pearson')
# df_xy.plot()
# df_xy.values

,CMA,Exchange,HML,RMW,Rm_Rf,SMB,y
count,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000
mean,-0.006765,-0.000848,-0.008217,0.010644,0.002328,-0.013002,0.006722
std,0.029092,0.008558,0.030692,0.042558,0.073005,0.054759,0.079759
min,-0.131520,-0.034406,-0.099511,-0.110696,-0.223365,-0.213241,-0.224896
25%,-0.016646,-0.005557,-0.020814,-0.013435,-0.040162,-0.033335,-0.034329
50%,-0.005898,-0.001542,-0.007342,0.006767,0.005440,-0.012613,0.000989
75%,0.008314,0.001744,0.008408,0.026979,0.036263,0.018532,0.036767
max,0.053428,0.028843,0.077680,0.156627,0.277888,0.151607,0.342380


In [ ]:
### 5

In [ ]:
import statsmodels.formula.api as smf
import statsmodels.api as sm

# 5个因子
ols_model = smf.ols('y ~ HML + Rm_Rf + SMB + CMA + RMW', data=df_xy)
results = ols_model.fit()#cov_type='HC3'
results.summary()